In [436]:
import os
import requests
import json
import pandas as pd
import subprocess

from spellchecker import SpellChecker

#print(os.getcwd())
#print(os.path.dirname(os.path.realpath('__file__')))
#print(os.listdir(os.path.dirname(os.path.realpath('__file__'))))

spell = SpellChecker()
spell.word_frequency.load_text_file('./words.txt')


# find those words that may be misspelled
#misspelled = spell.unknown(['let', 'us', 'wlak','on','the','groun']) 

#string_to_be_checked = "ngixn"
#spell.check(string_to_be_checked)


misspelled = spell.unknown(['nginx1', 'nignx', 'ngixn', 'nnginx'])
for word in misspelled:
    # Get the one `most likely` answer
    print("it should be : " + spell.correction(word))
    
    # Get a list of `likely` options
    #print(spell.candidates(word))


it should be : nginx
it should be : nginx
it should be : nginx
it should be : nginx


In [437]:
!ls -la

total 24
drwxr-xr-x. 3 jovyan users  123 Sep 15 07:09 .
drwxr-xr-x. 8 jovyan users  209 Sep 14 07:22 ..
drwxr-xr-x. 2 jovyan users  113 Sep 14 07:19 .ipynb_checkpoints
-rw-r--r--. 1 jovyan users 3350 Sep 15 03:39 monitoring.ipynb
-rw-r--r--. 1 jovyan users 1257 Sep 15 03:29 set_env.ipynb
-rw-r--r--. 1 jovyan users 8901 Sep 15 07:09 test_monitoring.ipynb
-rw-r--r--. 1 jovyan users    6 Sep 14 06:51 words.txt


In [438]:
prom_query_range_url = 'http://13.125.63.249:30915/api/v1/query_range?query='
prom_query_url = 'http://13.125.63.249:30915/api/v1/query?query='

a = prom_query_url + 'ALERTS{alertname="KubeContainerWaiting"}'


print(a)

http://13.125.63.249:30915/api/v1/query?query=ALERTS{alertname="KubeContainerWaiting"}


In [439]:
def call_data(url, query):
    r = requests.get(url + query)
    return r

In [440]:
def kube_container_waiting():
    alert_query = 'ALERTS{alertname="KubeContainerWaiting"}'
    alert_get = call_data(prom_query_url, alert_query)
    alert_message = json.loads(alert_get.text)
    
    fix_list = []
    for item in alert_message['data']['result']:
        #print(item)
        check_list = {'namespace':item['metric']['namespace'], 'pod':item['metric']['pod']}
        #check_list = {item['metric']['namespace'], item['metric']['pod']}
        print(check_list)        
        fix_list.append(check_list)     
    
    return(fix_list) 
    

In [441]:
def kubectl_jsonpath():
    pending_data = kube_container_waiting()
    cnt = len(pending_data)    
    df_data = pd.DataFrame(pending_data)
    #print(df_data)
    
    kubectl_result = []
    for i in range(cnt):
        namespace = df_data.namespace[i]
        pod = df_data.pod[i]
        
        cli = "kubectl -n " + namespace + " get pods " + pod + \
              " -o jsonpath='{.status.containerStatuses[*].state.waiting.message}'" + \
              " | awk -F'\"' '{print $2}'"
        
        kubectl_command_line = subprocess.check_output(cli, shell=True,  encoding='utf-8')
        #print(type(kubectl_command_line))
        #print(kubectl_command_line)
        tmp_string = namespace + " " + pod + " " + kubectl_command_line
        #print(tmp_string)
        
        ### sort :: namespace pod result        
        kubectl_result.append(tmp_string)
        
    return(kubectl_result)

In [460]:
def recommand_image():
    pending_images = kubectl_jsonpath()
    cnt = len(pending_images)
    spell = SpellChecker()
    spell.word_frequency.load_text_file('./words.txt')
    
    #result = []
    for i in range(cnt):
        namespace = pending_images[i].split(' ')[0]
        pod = pending_images[i].split(' ')[1]   
        m_image = pending_images[i].split(' ')[2]
        
        #misspelled = spell.unknown([m_image])
        tmp_recommend = spell.correction(m_image)
        tmp_string = "namespace: " + namespace \
                   + " , pod: " + pod \
                   + "\n" + " =====> Please Check your Pull Image. It should be : " + tmp_recommend
        result = tmp_string
        print(result)
        
    #return(result)
    

In [461]:
if __name__=="__main__":
    #print(kube_container_waiting())
    #print(kubectl_jsonpath())
    print(recommand_image())

{'namespace': 'aiops', 'pod': 'test-pending'}
{'namespace': 'default', 'pod': 'test-pending-de'}
namespace:default , pod:test-pending-de Please Check your Pulll Image. It should be : nginx


In [444]:
!kubectl -n aiops get pods test-pending -o jsonpath='{.status.containerStatuses[*].state.waiting.message}' | awk -F'"' '{print $2}'

nginx1


In [ ]:
a = !kubectl -n aiops get pods test-pending -o jsonpath='{.status.containerStatuses[*].state.waiting.message}'

In [ ]:
print(a)

In [376]:
b = !kubectl -n aiops get pods test-pending -o jsonpath='{.status.containerStatuses[*].state.waiting.message}' | awk -F"\"" '{print $2}'

In [380]:
print(b)

['nginx1']
